# Train pytorch model on CIFAR10 with Azure Machine Learning
Based on https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-train

### Workspace

In [ ]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py
from azureml.core import Workspace
ws = Workspace.from_config()

### Compute target
Create a compute target in the Azure Machine Learning studio.
- e.g. with the name 'cl-gpu-max1-pvt'
- 1 node GPU 
- idle time 360 seconds

In [ ]:
_compute_target = 'cl-gpu-max1-pvt'

### Change idle time of compute target
The inactive cluster will shut down after the idle time of seconds and will not incur significant costs.

In [ ]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py
from azureml.core import ComputeTarget

# choose idle times, i.e. 3600 or 360
ComputeTarget(ws, _compute_target).update(idle_seconds_before_scaledown=360) 

### Environment

In [ ]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py
from azureml.core import Environment

# use curated pytorch environment
env = Environment.get(workspace=ws, name="AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu")

# Or define custom environment (Note this will take quite some time to build the docker image the first time)
#env = Environment.from_conda_specification(
#    name='pytorch-env',
#    file_path='../pytorch-env.yml'
#)

### Experiment

In [ ]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py 
from azureml.core import Experiment
from azureml.core import ScriptRunConfig

experiment = Experiment(workspace=ws, name='CIFAR10')
config = ScriptRunConfig(source_directory='../code',
                            script='train.py',
                            arguments=['--debug', 0, '--numepochs', 2],
                            compute_target=_compute_target,
                            environment = env)

run = experiment.submit(config)
run.get_portal_url()